# Apache Beam - BigQuery

BigQuery is Google's cloud based Data Warehouse.  BigQuery can be used as a source or sink of data in a Beam pipeline.  In this notebook we will explore some of the BigQuery capabilities available in Beam.


First, we define the dependencies that we wish to load from the Maven repositories.

See also:
* [Google BigQuery I/O connector](https://beam.apache.org/documentation/io/built-in/google-bigquery/)
* [JavaDoc: BigQueryIO](https://beam.apache.org/releases/javadoc/2.43.0/org/apache/beam/sdk/io/gcp/bigquery/BigQueryIO.html)

Since our notebook is going to use Google Cloud SDK JARS we must include these in our dependencies.  Specifically, we need to include:

```
<dependency>
  <groupId>org.apache.beam</groupId>
  <artifactId>beam-sdks-java-io-google-cloud-platform</artifactId>
  <version>2.43.0</version>
</dependency>
```

Normally we would load our dependencies using the IJava Jupyter cell magic called `%%loadFromPom`.  Unfortunately, this doesn't work ([issue](https://github.com/SpencerPark/IJava/issues/139)).  A workaround is to download the dependencies outside of Jupyter and then launch Jupyter with the downloaded dependencies in the classpath.

```
mvn dependency:copy-dependencies
export IJAVA_CLASSPATH="./target/dependency/*"
jupyter notebook

```

Next we define our imports required for execution.

In [1]:
import java.util.Arrays;
import java.util.List;

import org.apache.beam.sdk.Pipeline;
import org.apache.beam.sdk.options.Default;
import org.apache.beam.sdk.options.Description;
import org.apache.beam.sdk.options.PipelineOptionsFactory;
import org.apache.beam.sdk.options.PipelineOptions;
import org.apache.beam.sdk.options.StreamingOptions;
import org.apache.beam.sdk.transforms.Create;
import org.apache.beam.sdk.values.PCollection;
import org.apache.beam.sdk.transforms.DoFn;
import org.apache.beam.sdk.transforms.ParDo;
import org.apache.beam.sdk.coders.KvCoder;
import org.apache.beam.sdk.coders.StringUtf8Coder;
import org.apache.beam.sdk.values.KV;
import org.apache.beam.sdk.io.gcp.bigquery.BigQueryIO;
import org.apache.beam.sdk.transforms.Sample;
import org.apache.beam.sdk.transforms.SerializableFunction;
import org.apache.beam.sdk.io.gcp.bigquery.SchemaAndRecord;
import org.apache.beam.sdk.io.gcp.bigquery.BigQueryIO.Write.WriteDisposition;
import org.apache.beam.sdk.io.gcp.bigquery.BigQueryIO.Write.CreateDisposition;
import com.google.api.services.bigquery.model.TableSchema;
import com.google.api.services.bigquery.model.TableFieldSchema;
import com.google.api.services.bigquery.model.TableRow;
import org.apache.beam.sdk.io.gcp.bigquery.TableRowJsonCoder;
import org.apache.beam.sdk.transforms.MapElements;
import org.apache.beam.sdk.values.TypeDescriptor;
import com.google.api.services.bigquery.model.TableReference;
import org.apache.beam.sdk.io.gcp.bigquery.InsertRetryPolicy;

String args[] = new String[] {"--tempLocation=gs://kolban-tmp"};
var options = PipelineOptionsFactory.fromArgs(args).withValidation().create();

## Querying and reading
Let us now read a table.  You must first create the table that you want to read from.  We will use a Google provided sample that is reasonably small.  The table is called `bigquery-public-data.samples.shakespeare` and contains:

* `word:STRING`
* `word_count:INTEGER`
* `corpus:STRING`
* `corpus_date:INTEGER`

The table contains 164K rows and appears to be about 6.13MB in size.

In this first example, we read a table fully.  The return is a PCollection of `TableRow` where each element corresponds to a single row in the table.

**Note**: The [TableRow](https://www.javadoc.io/static/com.google.apis/google-api-services-bigquery/v2-rev20220827-2.0.0/com/google/api/services/bigquery/model/TableRow.html) is a container meaning that it contains name/value pairs.  Each name corresponds to a column and the value to the cell in the table.  We can retrieve a value using the `get(name)` method:

```
TableRow row=...;
String word = row.get("word")
```

In [2]:
public class LoggingDoFn<T> extends DoFn<T, T>  {
  @ProcessElement
  public void processElement(@Element T element, OutputReceiver<T> out) {
    System.out.println(element);
    out.output(element);
  }
}

var tableName = "bigquery-public-data:samples.shakespeare";
var pipeline = Pipeline.create(options);
pipeline
  .apply("Read table", BigQueryIO.readTableRows().from(tableName))
  .apply("Sample", Sample.any(5))  // Just take 5 of the rows at random
  .apply("Print elements", ParDo.of(new LoggingDoFn<>()));
pipeline.run().waitUntilFinish();

GenericData{classInfo=[f], {word=LVII, word_count=1, corpus=sonnets, corpus_date=0}}
GenericData{classInfo=[f], {word=augurs, word_count=1, corpus=sonnets, corpus_date=0}}
GenericData{classInfo=[f], {word=dimm'd, word_count=1, corpus=sonnets, corpus_date=0}}
GenericData{classInfo=[f], {word=plagues, word_count=1, corpus=sonnets, corpus_date=0}}
GenericData{classInfo=[f], {word=treason, word_count=1, corpus=sonnets, corpus_date=0}}


DONE

Instead of reading the whole table, we can execute a query.  In the following we'll return the most frequently encountered words.  Notice the requirement to add `usingStandardSql()`.

In [3]:
var pipeline = Pipeline.create(options);
pipeline
  .apply("Read table", BigQueryIO
    .readTableRows()
    .fromQuery("select * FROM `bigquery-public-data.samples.shakespeare` order by word_count desc limit 5")
    .withQueryPriority(BigQueryIO.TypedRead.QueryPriority.INTERACTIVE)
    .usingStandardSql())
  .apply("Print elements",ParDo.of(new LoggingDoFn<>()));
pipeline.run().waitUntilFinish();

GenericData{classInfo=[f], {word=the, word_count=995, corpus=hamlet, corpus_date=1600}}
GenericData{classInfo=[f], {word=the, word_count=942, corpus=coriolanus, corpus_date=1607}}
GenericData{classInfo=[f], {word=the, word_count=937, corpus=kinghenryv, corpus_date=1599}}
GenericData{classInfo=[f], {word=the, word_count=894, corpus=2kinghenryiv, corpus_date=1598}}
GenericData{classInfo=[f], {word=the, word_count=848, corpus=kingrichardiii, corpus_date=1592}}


DONE

There are multiple styles/methods that can be used to read data from BigQuery.  One is `EXPORT` where the table is exported to Cloud Storage in Avro and then read from Avro.  The other is `DIRECT_READ` where the BigQuery storage API is used.  In the following, we show us switching to `DIRECT_READ`.

In [4]:
var pipeline = Pipeline.create(options);
pipeline
  .apply("Read table", BigQueryIO
    .readTableRows()
    .fromQuery("select * FROM `bigquery-public-data.samples.shakespeare` order by word_count desc limit 5")
    .withQueryPriority(BigQueryIO.TypedRead.QueryPriority.INTERACTIVE)
    .usingStandardSql()
    .withMethod(BigQueryIO.TypedRead.Method.DIRECT_READ))
  .apply("Print elements", ParDo.of(new LoggingDoFn<>()));
pipeline.run().waitUntilFinish();

GenericData{classInfo=[f], {word=the, word_count=995, corpus=hamlet, corpus_date=1600}}
GenericData{classInfo=[f], {word=the, word_count=942, corpus=coriolanus, corpus_date=1607}}
GenericData{classInfo=[f], {word=the, word_count=937, corpus=kinghenryv, corpus_date=1599}}
GenericData{classInfo=[f], {word=the, word_count=894, corpus=2kinghenryiv, corpus_date=1598}}
GenericData{classInfo=[f], {word=the, word_count=848, corpus=kingrichardiii, corpus_date=1592}}


DONE

We have looked at retrieving data as `TableRows` but maybe we want to work with our own data.  An alternative mechanism is to supply a function which is called for each row returned.  The function is passed a [SchemaAndRecord](https://beam.apache.org/releases/javadoc/2.42.0/org/apache/beam/sdk/io/gcp/bigquery/SchemaAndRecord.html) instance and returns your own record structure.

In [5]:
class MyFn implements SerializableFunction<SchemaAndRecord, String> {
  public String apply(SchemaAndRecord input) {
    var record = input.getRecord();
    String result = record.get("word").toString() + ':' + record.get("word_count");
    return result;
  }
}

var pipeline = Pipeline.create(options);
pipeline
  .apply("Read table", BigQueryIO
    .read(new MyFn())
    .fromQuery("select * FROM `bigquery-public-data.samples.shakespeare` order by word_count desc limit 5")
    .withQueryPriority(BigQueryIO.TypedRead.QueryPriority.INTERACTIVE)         
    .usingStandardSql())
  .apply("Print elements", ParDo.of(new LoggingDoFn<>()));
pipeline.run().waitUntilFinish();

the:995
the:942
the:937
the:894
the:848


DONE

There are many other reading options we could consider.

* `withSelectedFields()` - If we are are reading the whole table without running a query, we can constrain which columns are returned.

## Writing to BigQuery
Now we turn our attention to writing to BigQuery.  When we write to BigQuery, either the target table exists or it does not.  We can control what we wish to do using the `withCreateDisposition()`.  Our choices are either:

* `CREATE_NEVER` - Fail the request as the table should have existed.
* `CREATE_IF_NEEDED` - Create the table if it doesn't already exist.  We must also specify the schema for the table that is to be created.

Since we are adding rows to a table, the next question becomes what we want to happen if there are existing rows.  We can specify this with the `withWriteDisposition()` function.  Our choices are:

* `WRITE_EMPTY` - Fail the request if the table is not empty.
* `WRITE_TRUNCATE` - Empty the table before writing new rows.
* `WRITE_APPEND` - Append the new rows into the table leaving the existing rows untouched.

When the actual rows are to be inserted into a table, there are a few methods available to us.

* load jobs (`FILE_LOADS`)
* streaming inserts (`STREAMING_INSERTS`)
* storage write (`STORAGE_WRITE_API`)

We can select which one we wish to use with the `writeMethod()` function.  The default for a bounded PCollection is `FILE_LOADS` while for an unbounded PCollection it is `STREAMING_INSERTS`.

In order to write to BigQuery, we need to think in terms of TableRow objects.  We can either pass in a `PCollection<TableRow>` or pass in a `PCollection<???>` and provide a `withFormatFunction`.  The `withFormatFunction` is a function that will be invoked for each element in the PCollection that must return a TableRow.  Basically, a converter to TableRow.


In [6]:
String targetTableName = "test1-305123:jupyter.test1";

TableSchema schema = new TableSchema().setFields(
  Arrays.asList(
    new TableFieldSchema().setName("name").setType("STRING"),
    new TableFieldSchema().setName("salary").setType("FLOAT64"),
    new TableFieldSchema().setName("tenure").setType("INT64")
  )
);

class Employee implements Serializable {
  private String name;
  private Double salary;
  private Integer tenure;
  
  public Employee(String name, Double salary, Integer tenure) {
    this.name = name;
    this.salary = salary;
    this.tenure = tenure;
  }
  
  public String getName() {
    return name;
  }
  
  public Double getSalary() {
    return salary;
  }
  
  public Integer getTenure() {
    return tenure;
  }
  
  public String toString() {
    return "name: " + name + ", salary: " + salary + ", tenure: " + tenure;
  }
}

class EmployeeToTableRow implements SerializableFunction<Employee, TableRow> {
  public TableRow apply(Employee employee) {
    TableRow r = new TableRow();
    r.set("name", employee.getName());
    r.set("salary", employee.getSalary());
    r.set("tenure", employee.getTenure());
    return r;
  }
}

var pipeline = Pipeline.create(options);
pipeline
  .apply("Create Rows", Create.
    of(
      new Employee("Neil", 50000.11, 48),
      new Employee("Sue", 75000.99, 12),
      new Employee("Bob", 45000.32, 6)
    )
  )
  .apply("Write table", BigQueryIO
    .<Employee>write()
    .to(targetTableName)
    .withFormatFunction(new EmployeeToTableRow())
    .withSchema(schema)
    .withTableDescription("My Test Table")
    .withMethod(BigQueryIO.Write.Method.STREAMING_INSERTS)
    .withCreateDisposition(CreateDisposition.CREATE_IF_NEEDED)
    .withWriteDisposition(WriteDisposition.WRITE_APPEND));

pipeline.run().waitUntilFinish();

DONE

There are many additional options that can be specified when we write to a BigQuery table:

* `withTableDescription` - Provide a description of the table.
* `withMethod` - How the rows are written to BigQuery.  Choices include FILE_LOADS, STORAGE_API_AT_LEAST_ONCE, STORAGE_WRITE_API, STREAMING_INSERTS
* `withoutValidation` - Don't perform validation on the BigQuery command
* `withLoadProjectId` - The Google Cloud project to be billed for the request

## Error Handling
What if something goes wrong with our BigQuery call?  How can we catch this and take remedial action?

The following pipeline fragment will fail.  It attemptes to insert rows into a table that doesn't exist and we have instructed the transform to *not* create the table.

To solve the puzzle, we notice that a BigQueryIO.write transform returns an instance of [WriteResult](https://beam.apache.org/releases/javadoc/2.42.0/org/apache/beam/sdk/io/gcp/bigquery/WriteResult.html).  This is **not** a PCollection.  The WriteResult object contains methods to retrieve PCollections that have been stored within it which indicate the errors encountered:

* `getFailedInserts(): PCollection<TableRow>`
* `getFailedInsertsWithErr(): PCollection<BigQueryInsertError>`
* `getFailedStorageApiInserts(): PCollection<BigQueryStorageApiInsertError>`
* `getSuccessfulInserts(): PCollection<TableRow>` - **Warning**: This appears to only be available for STREAMING_INSERTS.
* `getSuccessfulTableLoads(): PCollection<TableDestination>`

To engage some of these results, we need to add some extract options to the transform:

* `withExtendedErrorInfo()` is needed to have data available to `getFailedInsertsWithErr()`

The transform can perform some error handling by itself.  We get to control this through `withFailedInsertRetryPolicy` which can take one of:

* alwaysRetry
* neverRetry
* retryTransientErrors

It also appears that there are distinctions between error handling and how the data is inserted.  For example, when we tried to load NaN it worked with FILE_LOADS and STORAGE_WRITE_API but failed with STREAMING_INSERTS.

When using STREAMING_INSERTS the data will be converted into JSON for transmission.  This can result in some unexpected errors.  For example, sending a row with the values:

```
name="Bob", salary=NaN, tenure=6
```
doesn't appear to be possible because we can't express NaN (Not a Number) in JSON.  However if we use the insertion method of `STORAGE_WRITE_API` or `FILE_LOADS` it works.


In [7]:
String targetTableName_notpresent = "test1-305123:jupyter.not_here";

var pipeline = Pipeline.create(options);
var writeResult = pipeline
  .apply("Create Rows", Create.<Employee>
    of(
      new Employee("Neil", 50000.11, 48),
      new Employee("Sue", 75000.99, 12),
      new Employee(null, 3.2, 6)
    )
  )
  .apply("Print initial elements", ParDo.of(new LoggingDoFn<>()))
  .apply("Write table", BigQueryIO
    .<Employee>write()
    .to(targetTableName_notpresent)
    .withExtendedErrorInfo()
    .withFormatFunction(new EmployeeToTableRow())
    .withSchema(schema)
    //.withFailedInsertRetryPolicy(InsertRetryPolicy.retryTransientErrors())
    .withTableDescription("My Test Table")
    .withMethod(BigQueryIO.Write.Method.STORAGE_WRITE_API)
    .withCreateDisposition(CreateDisposition.CREATE_NEVER)
    .withWriteDisposition(WriteDisposition.WRITE_APPEND));

//writeResult.getFailedStorageApiInserts()
//  .apply("Print elements", ParDo.of(new LoggingDoFn<>()));
//writeResult.getSuccessfulInserts()
//  .apply("Print elements", ParDo.of(new LoggingDoFn<>()));

System.out.println("About to run pipeline:");
pipeline.run().waitUntilFinish();
System.out.println("Completed!")

About to run pipeline:
name: Neil, salary: 50000.11, tenure: 48
name: null, salary: 3.2, tenure: 6
name: Sue, salary: 75000.99, tenure: 12


EvalException: java.lang.RuntimeException: java.lang.RuntimeException: java.lang.RuntimeException: com.google.api.gax.rpc.PermissionDeniedException: io.grpc.StatusRuntimeException: PERMISSION_DENIED: Permission 'TABLES_UPDATE_DATA' denied on resource 'projects/test1-305123/datasets/jupyter/tables/not_here' (or it may not exist).

## Final Notes
* It has been found that the IJava Jupyter package (1.3) was last built back in 2018.  This package builds a *fat jar* meaning that its dependencies are bundled into it.  One of those dependencies was com.google.gson which is a JSON parsing library from Google.  Unfortunately, the code from 2018 has less capability than the code for today and the Beam BigQueryIO depends on that code.  This resulted in some very strange errors that were hard to diagnose.  A workaround was found where we patched the IJava build with the latest GSON code base taken from Maven.

## References
* [BigQueryIO Source on Github](https://github.com/apache/beam/tree/master/sdks/java/io/google-cloud-platform/src/main/java/org/apache/beam/sdk/io/gcp/bigquery)